In [32]:
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [33]:
dataset_dir="C:\\Users\\HP\\Documents\\PlantVillage"

In [34]:
image_size=(256,256)

In [35]:
batch_size=32

In [36]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, image_size)
            return img_to_array(image)
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

In [37]:
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set the path to your dataset directory
dataset_dir = "C:/Users/HP/Documents/PlantVillage"

# Define the image size
image_size = (256, 256)

# Define the batch size
batch_size = 32

# Function to convert image to array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, image_size)
            return img_to_array(image)
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Load the images and labels
image_list = []
label_list = []

print("Loading images...")
for root_folder in os.listdir(dataset_dir):
    if root_folder != ".DS_Store":
        root_folder_path = os.path.join(dataset_dir, root_folder)
        for plant_folder in os.listdir(root_folder_path):
            if plant_folder != ".DS_Store":
                plant_folder_path = os.path.join(root_folder_path, plant_folder)
                if os.path.isdir(plant_folder_path):  # Check if it's a directory
                    for image_file in os.listdir(plant_folder_path)[:200]:
                        if image_file.endswith(".jpg") or image_file.endswith(".JPG"):
                            image_path = os.path.join(plant_folder_path, image_file)
                            image_list.append(convert_image_to_array(image_path))
                            label_list.append(plant_folder)

print("Image loading completed.")


Loading images...
Image loading completed.


In [38]:
# Convert the image list and label list to numpy arrays
X = np.array(image_list, dtype=np.float16) / 255.0
y = np.array(label_list)

In [39]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Specify the dataset directory containing the image data
dataset_dir = "C:/Users/HP/Documents/PlantVillage"

# Initialize lists to store image data and labels
image_list = []
label_list = []

# Iterate over the dataset directory and load images
for plant_folder in os.listdir(dataset_dir):
    if plant_folder != ".DS_Store":
        plant_folder_path = os.path.join(dataset_dir, plant_folder)
        for image_file in os.listdir(plant_folder_path)[:200]:
            if image_file.endswith(".jpg") or image_file.endswith(".JPG"):
                image_path = os.path.join(plant_folder_path, image_file)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:
                        # Preprocess the image (resize, normalize, etc.)
                        image = cv2.resize(image, (256, 256))  # Resize the image to 256x256
                        image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

                        # Add the preprocessed image and label to the lists
                        image_list.append(image)
                        label_list.append(plant_folder)
                    else:
                        print(f"Unable to read image: {image_path}")
                except Exception as e:
                    print(f"Error processing image: {image_path} - {e}")

# Perform label binarization
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(label_list)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(image_list, y, test_size=0.2, random_state=42)

# Rest of the code for model training and evaluation
# ...

In [55]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Convert the image lists to NumPy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Load the pre-trained ResNet50 model (without the top classification layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Add custom classification layers on top of the pre-trained base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(label_binarizer.classes_), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy:', accuracy)

Epoch 1/10
54/54 [==============================] - 334s 6s/step - loss: 2.4486 - accuracy: 0.1209 - val_loss: 2.3230 - val_accuracy: 0.2135
Epoch 2/10
54/54 [==============================] - 307s 6s/step - loss: 2.2925 - accuracy: 0.1761 - val_loss: 2.2233 - val_accuracy: 0.1787
Epoch 3/10
54/54 [==============================] - 307s 6s/step - loss: 2.2146 - accuracy: 0.2162 - val_loss: 2.1605 - val_accuracy: 0.1879
Epoch 4/10
54/54 [==============================] - 309s 6s/step - loss: 2.1113 - accuracy: 0.2452 - val_loss: 2.1083 - val_accuracy: 0.2529
Epoch 5/10
54/54 [==============================] - 322s 6s/step - loss: 2.0534 - accuracy: 0.2870 - val_loss: 1.9558 - val_accuracy: 0.3828
Epoch 6/10
54/54 [==============================] - 340s 6s/step - loss: 1.9767 - accuracy: 0.3382 - val_loss: 1.9335 - val_accuracy: 0.3411
Epoch 7/10
54/54 [==============================] - 319s 6s/step - loss: 1.9153 - accuracy: 0.3597 - val_loss: 1.8457 - val_accuracy: 0.3735
Epoch 8/10
54

In [40]:
# Make predictions on new images
new_images = np.array(image_list[1:5])  # Use the first 10 images for demonstration
new_images = np.reshape(new_images, (-1, 256, 256, 3))
new_images = new_images.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

# Perform predictions
predictions = model.predict(new_images)

# Decode the predictions
predicted_labels = label_binarizer.inverse_transform(predictions)
print("Predicted Labels:")
for label in predicted_labels:
    print(label)

1/1 [==============================] - 3s 3s/step
Predicted Labels:
Apple__rust
Apple__rust
Apple__rust
Apple__rust


In [41]:
disease_labels = {
    "Tomato__Target_Spot": 0,
    "Tomato__Tomato_mosaic_virus": 1,
    "Tomato__Tomato_YellowLeaf__Curl_Virus": 2,
    # Add more disease classes here
}

In [42]:
label_list.append(disease_labels
                  [plant_folder])

In [43]:
model.save("C:/Users/HP/Documents/model.h5")

In [44]:
# Load the trained model
model = load_model("C:/Users/HP/Documents/model.h5")

# Preprocess the new image
new_image_path = "C:/Users/HP/Documents/PlantVillage/Tomato__Tomato_mosaic_virus/0a7cc59f-b2b0-4201-9c4a-d91eca5c03a3___PSU_CG 2230.jpg"
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (256, 256))
new_image = new_image.astype('float32') / 255.0
new_image = np.expand_dims(new_image, axis=0)  # Add an extra dimension for batch size

# Perform prediction
predictions = model.predict(new_image)

# Decode the predictions
predicted_labels = label_binarizer.inverse_transform(predictions)

print("Predicted Labels:", predicted_labels)

1/1 [==============================] - 2s 2s/step
Predicted Labels: ['Tomato__Tomato_mosaic_virus']


In [45]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("C:/Users/HP/Documents/model.h5")

In [46]:
new_image = cv2.imread(new_image_path)
if new_image is None:
    print(f"Unable to read image: {new_image_path}")
else:
    new_image = cv2.resize(new_image, (256, 256))
    new_image = new_image.astype('float32') / 255.0
    new_image = np.expand_dims(new_image, axis=0)  # Add an extra dimension for batch size

In [47]:
from PIL import Image

# Load image using Pillow (alternative to cv2.imread)
try:
    image = Image.open(new_image_path)
    # Perform resizing and preprocessing as needed
    # ...
except Exception as e:
    print(f"Unable to read image: {new_image_path} - {e}")

In [48]:
import cv2
import numpy as np

# Load the image
image_path = "C:/Users/HP/Documents/PlantVillage/Tomato__Tomato_mosaic_virus/0a7cc59f-b2b0-4201-9c4a-d91eca5c03a3___PSU_CG 2230.jpg"
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is not None:
    # Preprocess the image
    image = cv2.resize(image, (256, 256))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)  # Add an extra dimension for batch size

    # Make predictions using the model
    predictions = model.predict(image)

    # Decode the predictions
    predicted_labels = label_binarizer.inverse_transform(predictions)

    # Print the predicted labels
    print("Predicted Disease:", predicted_labels[0])
else:
    print("Unable to read the image:", image_path)

1/1 [==============================] - 2s 2s/step
Predicted Disease: Tomato__Tomato_mosaic_virus


In [49]:
from PIL import Image

# Load the image
image_path = "C:/Users/HP/Documents/PlantVillage/Tomato__Tomato_mosaic_virus/0a7cc59f-b2b0-4201-9c4a-d91eca5c03a3___PSU_CG 2230.jpg"
image = Image.open(image_path)

# Convert the image to RGB if it's in a different mode
if image.mode != "RGB":
    image = image.convert("RGB")

# Resize the image
image = image.resize((256, 256))

# Convert the image to a NumPy array
image = np.array(image)

# Preprocess the image
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=0)  # Add an extra dimension for batch size

# Make predictions using the model
predictions = model.predict(image)

# Decode the predictions
predicted_labels = label_binarizer.inverse_transform(predictions)

# Print the predicted labels
print("Predicted Disease:", predicted_labels[0])

1/1 [==============================] - 0s 249ms/step
Predicted Disease: Tomato__Tomato_mosaic_virus


In [64]:
import os
import cv2
import numpy as np

# Specify the path to the image file
image_path = "C:/Users/HP/Documents/PlantVillage/Tomato__Tomato_mosaic_virus/0a91f50b-1263-4b2c-a8c1-f2a6025b82f3___PSU_CG 2136.jpg"

# Check if the image file exists
if not os.path.isfile(image_path):
    print("Image file not found!")
else:
    try:
        # Read the image
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is None:
            print("Failed to load image!")
        else:
            # Preprocess the image
            image = cv2.resize(image, (256, 256))
            image = image.astype('float32') / 255.0
            image = np.expand_dims(image, axis=0)  # Add an extra dimension for batch size

            # Perform prediction using the loaded model
            predictions = model.predict(image)

            # Decode the predictions
            predicted_labels = label_binarizer.inverse_transform(predictions)
            print("Predicted label:", predicted_labels[0])
    except Exception as e:
        print("Error processing image:", str(e))

1/1 [==============================] - 0s 281ms/step
Predicted label: Tomato__Tomato_mosaic_virus


In [50]:
import os
import cv2
import numpy as np

# Define the image file path
image_file = "C:/Users/HP/Documents/PlantVillage/Tomato__Tomato_mosaic_virus/0a91f50b-1263-4b2c-a8c1-f2a6025b82f3___PSU_CG 2136.jpg"

# Check if the image file exists
if not os.path.isfile(image_file):
    print(f"Image file not found: {image_file}")
    exit()

# Load the image
try:
    image = cv2.imread(image_file)
    if image is None:
        print(f"Unable to read image file: {image_file}")
        exit()
except Exception as e:
    print(f"Error loading image file: {image_file} - {e}")
    exit()

# Preprocess the image
try:
    image = cv2.resize(image, (256, 256))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
except Exception as e:
    print(f"Error preprocessing image: {image_file} - {e}")
    exit()

# Perform prediction using the loaded model
try:
    predictions = model.predict(image)
    predicted_labels = label_binarizer.inverse_transform(predictions)
    print("Predicted Labels:", predicted_labels)
except Exception as e:
    print(f"Error performing prediction: {e}")
    exit()

1/1 [==============================] - 0s 258ms/step
Predicted Labels: ['Tomato__Tomato_mosaic_virus']


In [51]:
# Load the trained model
model = load_model("C:/Users/HP/Documents/model.h5")

In [ ]:
import os

import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Specify the dataset directory containing the image data
dataset_dir = "C:/Users/HP/Documents/PlantVillage"

# Initialize lists to store image data and labels
image_list = []
label_list = []

# Iterate over the dataset directory and load images
for plant_folder in os.listdir(dataset_dir):
    if plant_folder != ".DS_Store":
        plant_folder_path = os.path.join(dataset_dir, plant_folder)
        for image_file in os.listdir(plant_folder_path)[:200]:
            if image_file.endswith(".jpg") or image_file.endswith(".JPG"):
                image_path = os.path.join(plant_folder_path, image_file)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:
                        # Preprocess the image (resize, normalize, etc.)
                        image = cv2.resize(image, (299, 299))  # Resize the image to 299x299 for InceptionV3
                        image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

                        # Add the preprocessed image and label to the lists
                        image_list.append(image)
                        label_list.append(plant_folder)
                    else:
                        print(f"Unable to read image: {image_path}")
                except Exception as e:
                    print(f"Error processing image: {image_path} - {e}")

# Perform label binarization
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(label_list)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(image_list, y, test_size=0.2, random_state=42)

# Convert the image lists to NumPy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Load the pre-trained InceptionV3 model (without the top classification layer)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add custom classification layers on top of the pre-trained base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(label_binarizer.classes_), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy:', accuracy)

In [53]:
import os

import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [54]:
# Specify the dataset directory containing the image data
dataset_dir = "C:/Users/HP/Documents/PlantVillage"

In [55]:
# Initialize lists to store image data and labels
image_list = []
label_list = []

# Iterate over the dataset directory and load images
for plant_folder in os.listdir(dataset_dir):
    if plant_folder != ".DS_Store":
        plant_folder_path = os.path.join(dataset_dir, plant_folder)
        for image_file in os.listdir(plant_folder_path)[:200]:
            if image_file.endswith(".jpg") or image_file.endswith(".JPG"):
                image_path = os.path.join(plant_folder_path, image_file)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:
                        # Preprocess the image (resize, normalize, etc.)
                        image = cv2.resize(image, (299, 299))  # Resize the image to 299x299 for InceptionV3
                        image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

                        # Add the preprocessed image and label to the lists
                        image_list.append(image)
                        label_list.append(plant_folder)
                    else:
                        print(f"Unable to read image: {image_path}")
                except Exception as e:
                    print(f"Error processing image: {image_path} - {e}")

In [56]:
# Perform label binarization
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(label_list)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(image_list, y, test_size=0.2, random_state=42)

# Convert the image lists to NumPy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [57]:
# Load the pre-trained InceptionV3 model (without the top classification layer)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add custom classification layers on top of the pre-trained base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(label_binarizer.classes_), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy:', accuracy)

Epoch 1/5
54/54 [==============================] - 381s 7s/step - loss: 0.1607 - accuracy: 0.9541 - val_loss: 0.1728 - val_accuracy: 0.9350
Epoch 2/5
54/54 [==============================] - 418s 8s/step - loss: 0.0943 - accuracy: 0.9826 - val_loss: 0.1785 - val_accuracy: 0.9327
Epoch 3/5
54/54 [==============================] - 380s 7s/step - loss: 0.0612 - accuracy: 0.9959 - val_loss: 0.1680 - val_accuracy: 0.9188
Epoch 4/5
54/54 [==============================] - 362s 7s/step - loss: 0.0460 - accuracy: 0.9965 - val_loss: 0.1320 - val_accuracy: 0.9490
Epoch 5/5
14/14 [==============================] - 73s 5s/step - loss: 0.1836 - accuracy: 0.9397
Test Accuracy: 0.9396751523017883


In [61]:
print(new_images.shape)

(4, 299, 299, 3)


In [66]:
# Make predictions on new images
new_images = np.array(image_list[:5])  # Use the first 4 images for demonstration
new_images_resized = np.array([cv2.resize(image, (256, 256)) for image in new_images])
new_images_resized = new_images_resized.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

# Perform predictions
predictions = model.predict(new_images_resized)

# Decode the predictions
predicted_labels = label_binarizer.inverse_transform(predictions)
print("Predicted Labels:")
for label in predicted_labels:
    print(label)

1/1 [==============================] - 1s 829ms/step
Predicted Labels:
Apple__rust
Apple__rust
Apple__rust
Apple__rust
Apple__rust


In [71]:
import cv2
import numpy as np

# Define the image paths
image_paths = [
    "C:/Users/HP/Documents/PlantVillage/Potato___healthy/0f4ebc5a-d646-436a-919d-961342997cde___RS_HL 4183.jpg"]

# Load and preprocess the images
new_images = []
for image_path in image_paths:
    try:
        # Read the image
        image = cv2.imread(image_path)
        if image is not None:
            # Resize the image to (256, 256)
            image = cv2.resize(image, (256, 256))
            image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]
            new_images.append(image)
        else:
            print(f"Unable to read image: {image_path}")
    except Exception as e:
        print(f"Error processing image: {image_path} - {e}")

# Convert the list of images to a NumPy array
new_images = np.array(new_images)

# Perform predictions
predictions = model.predict(new_images)

# Decode the predictions
predicted_labels = label_binarizer.inverse_transform(predictions)
print("Predicted Labels:")
for label in predicted_labels:
    print(label)

1/1 [==============================] - 0s 258ms/step
Predicted Labels:
Potato___healthy
